In [45]:
import pymongo
import pandas as pd

In [46]:
game_dict= { 'NASA': {'user1':10,
                    'user2':7,
                    'user3':2},
           'STAR WARS': {'user2':5,
                        'user4':10,
                        'user5':2},
            'Mario': {'user1': 10,
                     'user3':8,
                     'user6':2}}

In [47]:
game_dict['STAR WARS']

{'user2': 5, 'user4': 10, 'user5': 2}

In [48]:
game_dict

{'NASA': {'user1': 10, 'user2': 7, 'user3': 2},
 'STAR WARS': {'user2': 5, 'user4': 10, 'user5': 2},
 'Mario': {'user1': 10, 'user3': 8, 'user6': 2}}

In [49]:
def flatten_game_dict(game_dict):
    for (game_id, user_score_dict) in game_dict.items():
        for (username, score) in user_score_dict.items():
            yield {'game_id': game_id, 'username': username, 'score': score}
            

In [50]:
flatten_game_dict(game_dict=game_dict)

<generator object flatten_game_dict at 0x115ef4af0>

In [51]:
mc=pymongo.MongoClient()

In [52]:
db=mc['practice_db']

In [53]:
practice=db['practice']

flattened = flatten_game_dict(game_dict=game_dict)

#for thing in flattened:
    practice.insert_one(thing)

In [54]:
practice.find_one()

{'_id': ObjectId('5c0964d7933dea443f61b922'),
 'game_id': 'NASA',
 'username': 'user1',
 'score': 10}

In [56]:
def load_reviews_for_game(key, game_id):
    found_reviews = practice.find({key: game_id})
    return found_reviews

In [57]:
load_reviews_for_game('username', 'user1').distinct('game_id')

['NASA', 'Mario']

In [58]:
df=pd.DataFrame(list(practice.find()))

In [59]:
df

,_id,game_id,score,username
0,5c0964d7933dea443f61b922,NASA,10,user1
1,5c0964d7933dea443f61b923,NASA,7,user2
2,5c0964d7933dea443f61b924,NASA,2,user3
3,5c0964d7933dea443f61b925,STAR WARS,5,user2
4,5c0964d7933dea443f61b926,STAR WARS,10,user4
5,5c0964d7933dea443f61b927,STAR WARS,2,user5
6,5c0964d7933dea443f61b928,Mario,10,user1
7,5c0964d7933dea443f61b929,Mario,8,user3
8,5c0964d7933dea443f61b92a,Mario,2,user6


In [83]:
for use in df.username.unique():
    print(use)

user1
user2
user3
user4
user5
user6


In [84]:
df[df.username=='user1']

,_id,game_id,score,username
0,5c06d1d5933dea14d5c2aad3,NASA,10,user1
6,5c06d1d6933dea14d5c2aad9,Mario,10,user1


In [60]:
preferences=df.pivot(index='username', columns='game_id', values='score' )

In [61]:
preferences

game_id,Mario,NASA,STAR WARS
username,,,
user1,10.0,10.0,NaN
user2,NaN,7.0,5.0
user3,8.0,2.0,NaN
user4,NaN,NaN,10.0
user5,NaN,NaN,2.0
user6,2.0,NaN,NaN


In [63]:
users=set(df['username'])
games=set(df['game_id'])

In [65]:
game_id_lookup = dict(zip(games, range(len(games))))
user_id_lookup = dict(zip(users, range(len(users))))

In [73]:
df['game_number']=df['game_id'].apply(game_id_lookup.get)
df['user_number']=df['username'].apply(user_id_lookup.get)

In [74]:
df

,_id,game_id,score,username,game_number,user_number
0,5c0964d7933dea443f61b922,NASA,10,user1,2,4
1,5c0964d7933dea443f61b923,NASA,7,user2,2,0
2,5c0964d7933dea443f61b924,NASA,2,user3,2,5
3,5c0964d7933dea443f61b925,STAR WARS,5,user2,0,0
4,5c0964d7933dea443f61b926,STAR WARS,10,user4,0,2
5,5c0964d7933dea443f61b927,STAR WARS,2,user5,0,3
6,5c0964d7933dea443f61b928,Mario,10,user1,1,4
7,5c0964d7933dea443f61b929,Mario,8,user3,1,5
8,5c0964d7933dea443f61b92a,Mario,2,user6,1,1


In [75]:
preferences=df.pivot(index='user_number', columns='game_number', values='score' )

In [76]:
preferences

game_number,0,1,2
user_number,,,
0,5.0,NaN,7.0
1,NaN,2.0,NaN
2,10.0,NaN,NaN
3,2.0,NaN,NaN
4,NaN,10.0,10.0
5,NaN,8.0,2.0
